<a href="https://colab.research.google.com/github/bylehn/auxetic_networks_jaxmd/blob/stress-response/auxetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
#!pip install jax-md
import jax.numpy as np
import numpy as onp
from jax import random
from jax.config import config; config.update("jax_enable_x64", True)
from jax_md import space
from jax import random, grad
from jax import jit, vmap
from jax import lax
import networkx as nx
import visualize
import utils
import simulation


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
steps = 50
write_every = 1
perturbation = 1.0
delta_perturbation = 0.1
#create graph

number_of_nodes_per_side = 10
N,G,X,E,bond_lengths = utils.createDelaunayGraph(number_of_nodes_per_side, 22, 2.0, 0.2)
R = np.array(X)
k_angle = 1e-1
k_bond, _ = utils.create_spring_constants(R,E,1.0)
surface_nodes = utils.getSurfaceNodes(G, number_of_nodes_per_side)
displacement, shift = space.free() #displacement = points in space, shift = small shifts of each particle
angle_triplets_data = utils.calculate_angle_triplets(E)
theta_0 = utils.calculate_initial_angles(R, displacement, E, angle_triplets_data)
grad_f = grad(simulation.simulate_auxetic, argnums=1)

In [3]:
poisson, log, R_init, R_final = simulation.simulate_auxetic(R, k_bond, k_angle, shift, surface_nodes, perturbation, delta_perturbation, displacement, E, bond_lengths, theta_0, steps, write_every, optimize=False)